In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error

from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX

from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

import psutil
import ast

In [2]:
sns.set_style("white", {'axes.grid' : False})
sns.set_palette("tab20", 16)
sns.set(font="Calibri")
sns.set_context('talk')

# Running the  model for each dataset

# FUNCTIONS

In [4]:
hourly_data = pd.read_csv('hourly_data_AUG_19.csv', parse_dates=['time'])

In [5]:
hourly_data

,channel,time,pm_2_5
0,aq_04,2019-01-12 14:00:00+03:00,24.143333
1,aq_04,2019-01-12 15:00:00+03:00,21.483333
2,aq_04,2019-01-12 16:00:00+03:00,NaN
3,aq_04,2019-01-12 17:00:00+03:00,NaN
4,aq_04,2019-01-12 18:00:00+03:00,NaN
...,...,...,...
165775,aq_49,2019-09-02 13:00:00+03:00,22.874000
165776,aq_49,2019-09-02 14:00:00+03:00,19.804444
165777,aq_49,2019-09-02 15:00:00+03:00,29.308222
165778,aq_49,2019-09-02 16:00:00+03:00,21.411429


In [ ]:
def full_days_only(d):
    # Interpolating gaps within the data
    d = d.set_index('time')
    d = d.interpolate(method='time');
#     print(d)
    # drop any remaining nans from the start
    d =  d.dropna().reset_index();
    # Setting first complete day and last complete day to assist with training quality
    first_full_day = pd.to_datetime(d.loc[d.time.dt.hour == 0.00, 'time'][:1].values[0], utc=True)
    last_full_day = pd.to_datetime(d.loc[d.time.dt.hour == 23.00, 'time'][-1:].values[0], utc=True)  
    # THen correct Kampala values are applied
    d_whole_days = d.loc[(d.time >= first_full_day) & (d.time <= last_full_day)]
    d_whole_days = d_whole_days.set_index('time')
#     print(d_whole_days)
    return d_whole_days

In [ ]:
def out_of_sample(d, oos_size):
    df = d.iloc[:-oos_size, 1]
    oos = d.iloc[-oos_size:, 1]
#     print('df.shape', df.shape)
#     print('oos.shape', oos.shape)
    return df, oos

In [ ]:
# Creating a list of channels to iterate through
chanlist = hourly_data.channel.unique().tolist()

In [ ]:
def plot_acf_pcf(df, lags):
    # plots specifying number of lags to includ
    plt.figure(figsize=(20,6))
    # acf
    axis = plt.subplot(2, 1, 1)
    plot_acf(df, ax=axis, lags=lags)
    # pacf
    axis = plt.subplot(2, 1, 2)
    plot_pacf(df, ax=axis, lags=lags)
    # show plot
#     print(chan)
    plt.show()

In [ ]:
for chan in chanlist:
    d = hourly_data.loc[hourly_data.channel == chan, ['time','pm_2_5']]
#     print(d)
    df = full_days_only(d)
    print('Channel: ', chan)
    # USE THIS IF WANT TO GENERATE AUTOCORRELATION GRAPHS
#     plot_acf_pcf(df, 24)

In [ ]:
# # Generating a dataframe for each channel
# for chan in chanlist:
#     # selecting only rows relating to the given channel
#     d = data_grp_hourly.loc[data_grp_hourly.channel == chan]
# #     print('d', d)
#    # removing partial days at start and end of sample 
#     d = full_days_only(d)
# #     print('d', d)
#     # set size of out of sample test data
#     oos_size = 24
#     df, oos = out_of_sample(d, oos_size)
    
#     lags = 24
#     plot_acf_pcf(df, lags)


# Preparing data for forecast model

In [14]:
# split a univariate dataset into train/test sets
def split_dataset(data):
    # split into standard weeks with 1 week validation and 1 day oos test
    train, test = data[0:-192], data[-192:-24]
    final_test = data[-24:]
    # restructure into windows of weekly data
    train = array(split(train, (len(train)/24)))
    test = array(split(test, (len(test)/24)))
    print(train, test, final_test)
    return train, test, final_test

In [15]:
# convert windows of weekly multivariate data into a series of total power
def to_series(data):
    # extract just the total power from each week
    series = [day for day in data]
    # flatten into a single series
    series = array(series).flatten()
#     print('series.shape: ', series.shape)
    return series

In [16]:
# sarima forecast
# Takes the history (train set plus day by day testing) and configuration
# converts history values to a single long series
# generates the sarima model based on config parameters
# fits the sarima model to the series data
# creates yhat, a prediction of the next 24 hours int he test set
def sarima_forecast(history, config):
    order, sorder, trend = config
#     print('order', order)
#     print('sorder', sorder)
#     print('trend', trend)
    # convert history into a univariate series
    series = to_series(history)
#     series = to_series(history)
#     print('series', series)
    # define model
    model = SARIMAX(series, order=order, seasonal_order=sorder, trend = trend,enforce_stationarity=False, enforce_invertibility=False)
#     model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
#     print('model', model)
    # fit model
    model_fit = model.fit(disp=False)
    # make one step forecast
    yhat = model_fit.predict(len(series), len(series)+23)
#     print('yhat', yhat)
#     print('yhat 0', yhat[0])
    return yhat

In [17]:
# evaluate a single model which creates a prediction for ech day and each hour
# This is then fed into the evaluate forecast function to generate overall scores for the model
# for the model
# This needs to happen for every incarnation of the model
def evaluate_model(model_func, train, test, config):
    # history is a list of weekly data
    history = [x for x in train]
    # walk-forward validation over each week
    predictions = list()
    for i in range(len(test)):
        # predict the week
##        config = ((2,0,6), (1,0,1,24), 'n')
        yhat_sequence = model_func(history, config)
#         yhat_sequence = sarima_fore(history, config)
#         print('history', history)
#         print('yhat_sequence', yhat_sequence)
        # store the predictions
        predictions.append(yhat_sequence)
        # get real observation and add to history for predicting the next week
#         print('test[i,:]',test[i,:])
        history.append(test[i, :])
#         print('predictions', predictions)
#         print('history', history)
    predictions = array(predictions)
#     print('len(predictions)', len(predictions))
#     print('predictions.shape', predictions.shape)
    # evaluate predictions days for each week
#     score, scores = evaluate_forecasts(test, predictions)
    score, scores = evaluate_forecasts(test, predictions)
    print('score', score)
    print('scores', scores)
#     return score, scores
    return score, scores

In [18]:
def evaluate_forecasts(actual, predicted):
#     print('actual.shape : ', actual.shape)
#     print('predicted.shape', predicted.shape)
    scores = list()
    # calculate an RMSE score for each day
    for i in range(actual.shape[1]):
        # calculate mse
        print('i', i)
#         print('actual.shape : ', actual.shape)
#         print('predicted.shape', predicted.shape)
        mse = mean_squared_error(actual[:, i], predicted[:, i])
        print('mse', mse)
        # calculate rmse
        rmse = sqrt(mse)
        print('rmse', rmse)
        # store
        scores.append(rmse)

    # calculate overall RMSE
#     count +=1
#     print('COUNT: ', count)
    s = 0
    for row in range(actual.shape[0]):
        for col in range(actual.shape[1]):
            s += (actual[row, col] - predicted[row, col])**2
    score = sqrt(s / (actual.shape[0] * actual.shape[1]))
#     return score, scores
    print('score, scores: ', score, scores)

    return score, scores

In [19]:
# summarize scores
#Takes the name, model score and list of hourley mean scores
#print
def summarize_scores(name, score, scores):
    s_scores = ', '.join(['%.1f' % s for s in scores])
    print('%s: [%.3f]' % (name, score))

In [20]:
# create a set of sarima configs to try
def sarima_configs():
    models = list()
    # define config lists
#     p_params = [0,1,2]
    p_params = [2]
    d_params = [0]
#     q_params = [0,1,2]        
    q_params = [24]

#     t_params = ['n','c','t','ct']
    t_params = ['c']
    P_params = [0,1]
    D_params = [0]
    Q_params = [0,1]
    m_params = [24]
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    models.append(cfg)
    return models

In [21]:
print(sarima_configs())

[[(2, 0, 24), (0, 0, 0, 24), 'c'], [(2, 0, 24), (0, 0, 1, 24), 'c'], [(2, 0, 24), (1, 0, 0, 24), 'c'], [(2, 0, 24), (1, 0, 1, 24), 'c']]


In [22]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [23]:
# grid search configs
# Using train, test data and the list of configurations
# working in parallel


# def grid_search(data, cfg_list, n_test, parallel=True):
def grid_search(model_func, train, test, cfg_list, parallel=True):
    scores = None
    if parallel:
        # execute configs in parallel
        executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing', verbose=1)
        tasks = (delayed(score_model)(model_func,train, test, cfg) for cfg in cfg_list)
        scores = executor(tasks)
    else:
        scores = [score_model(model_func,train, test, cfg) for cfg in cfg_list]
    # remove empty results
    print(scores)
    scores = [r for r in scores if r[1] != None]
    # sort configs by error, asc
    scores.sort(key=lambda tup: tup[1])
    return scores

In [24]:
# score a model, return None on failure

def score_model(model_func,train, test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
#         result = walk_forward_validation(data, n_test, cfg)
        result = evaluate_model(model_func,train, test, cfg)[0]
    else:
        # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = evaluate_model(model_func,train, test, cfg)[0]
        except:
            error = None
    # check for an interesting result
    if result is not None:
        print(' > Model[%s] %.3f' % (key, result))
    return (key, result)

In [ ]:
# Generating a dataframe for each channel
for chan in chanlist[5:10]:
#     # selecting only rows relating to the given channel
#     d = data_grp_hourly.loc[data_grp_hourly.channel == chan]
        # selecting only rows relating to the given channel
    d = hourly_data.loc[hourly_data.channel == chan]
#     print(d)
#    # removing partial days at start and end of sample
#     df = df.reset_index()
    df = full_days_only(d)
#     print('d', d)
    # set size of out of sample test data
    oos_size = 24
    df, oos = out_of_sample(df, oos_size)

    # Generating train and test
    train, test, final_test = split_dataset(df)[0:3]
    
    #Checking the size of the train and test sets
#     print('train shape: ', train.shape)
#     print('test shape: ', test.shape)
#     print('final test shape',final_test.shape)
    
    # define the names and functions for the models we wish to evaluate
    models = dict()
    models['sarima'] = sarima_forecast

    # name each hour to be aggregated
    hours = ['0', '1', '2', '3', '4', '5', '6', '7', '8','9','10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
    print('channel', chan)
    # data split
    n_test = 24
    # model configs
    cfg_list = sarima_configs()
    # print(cfg_list)
    # grid search
    count=0
    scores = grid_search(sarima_forecast, train, test, cfg_list)
    print('channel: '+str(chan) +'done')
    # list top 3 configs
    for cfg, error in scores[:5]:
        print(cfg, error)
    print(scores)    
    # best_config = scores[:1]
    best_config = ast.literal_eval(scores[:1][0][0])
    print('best config', best_config)
    best_oos_yhat = sarima_forecast(df, best_config)
    oos_rmse = measure_rmse(final_test, best_oos_yhat)
    print('Out of sample rmse: ', oos_rmse)

[[ 39.60407025  40.0488843   40.49369835 ...  48.94516529  49.38997934
   49.83479339]
 [ 50.27960744  50.72442149  51.16923554 ...  69.23851064 129.00787234
   44.66723404]
 [ 40.86727273  54.5755      48.16590909 ...  87.83967742  63.92391304
   75.94347826]
 ...
 [ 49.78319149  49.88295455  46.61421053 ... 132.35531915  94.89065217
   74.96704545]
 [ 53.35        51.26978723  50.6837037  ... 113.70808511  95.2806383
  103.46723404]
 [ 80.50680851  53.58446809  87.44489362 ...  76.86742424  76.1580303
   75.44863636]] [[ 74.73924242  74.02984848  73.32045455  72.61106061  71.90166667
   71.19227273  70.48287879  69.77348485  69.06409091  68.35469697
   67.64530303  66.93590909  66.22651515  65.51712121  64.80772727
   64.09833333  63.38893939  62.67954545  61.97015152  61.26075758
   60.55136364  59.8419697   59.13257576  58.42318182]
 [ 57.71378788  57.00439394  56.295       55.58560606  54.87621212
   54.16681818  53.45742424  52.7480303   52.03863636  51.32924242
   50.61984848  4

[Parallel(n_jobs=64)]: Using backend MultiprocessingBackend with 64 concurrent workers.


i 0
mse 81.7350500166288
rmse 9.040743886242371
i 1
mse 154.10567713021652
rmse 12.413930768705637
i 2
mse 107.93327041456998
rmse 10.389093820664533
i 3
mse 143.74182254620513
rmse 11.98923778003444
i 4
mse 121.53700445676786
rmse 11.024382270983162
i 5
mse 167.11303857461326
rmse 12.927220837233858
i 6
mse 135.73050243511372
rmse 11.650343447088318
i 7
mse 247.10483631667938
rmse 15.719568579216142
i 8
mse 229.9036760723326
rmse 15.162574849686072
i 9
mse 191.05467662696432
rmse 13.82225295047679
i 10
mse 177.1854454803978
rmse 13.311102339040062
i 11
mse 215.51603092530448
rmse 14.680464261231812
i 12
mse 277.4489125713182
rmse 16.656797788630268
i 13
mse 374.891876126612
rmse 19.362124783365385
i 14
mse 494.8232046789373
rmse 22.244621927084697
i 15
mse 429.8405166504473
rmse 20.73259551166827
i 16
mse 209.35888421449653
rmse 14.469239241041546
i 17
mse 82.73701664747306
rmse 9.095989041741039
i 18
mse 167.7992840279246
rmse 12.953736296062408
i 19
mse 683.1854054092403
rmse 26.137

[Parallel(n_jobs=64)]: Done   2 out of   4 | elapsed: 15.9min remaining: 15.9min


i 0
mse 80.50903113495671
rmse 8.972682493822944
i 1
mse 146.7014673640448
rmse 12.112038117676347
i 2
mse 101.69182821556572
rmse 10.084236620367736
i 3
mse 135.71949647080825
rmse 11.649871092454553
i 4
mse 120.42837399241479
rmse 10.973986239849893
i 5
mse 176.42400885186257
rmse 13.282469983096615
i 6
mse 157.9745961685954
rmse 12.568794539198873
i 7
mse 224.1443175784671
rmse 14.971450082689623
i 8
mse 234.46746117621603
rmse 15.31233036399803
i 9
mse 191.28756862382448
rmse 13.830674915701854
i 10
mse 174.25854311535113
rmse 13.200702372046388
i 11
mse 215.51123287422038
rmse 14.68030084413192
i 12
mse 284.70430169172033
rmse 16.87318291525699
i 13
mse 375.08694215753275
rmse 19.36716143779291
i 14
mse 482.1198621612882
rmse 21.957228016334124
i 15
mse 428.0219853855063
rmse 20.68869221061366
i 16
mse 215.25603557564517
rmse 14.671606441547059
i 17
mse 84.00638496995076
rmse 9.16549971196065
i 18
mse 156.26982932787823
rmse 12.500793147951782
i 19
mse 616.8135840909538
rmse 24.83

...

/home/jupyter/.local/lib/python3.5/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Out of sample rmse:  26.732863084704157
[[ 44.60489316  44.81987179  45.03485043 ...  49.11944444  49.33442308
   49.54940171]
 [ 49.76438034  49.97935897  50.19433761 ... 201.29020833 146.25833333
  110.551875  ]
 [109.91642857 104.47515873  99.03388889 ... 187.659375   169.81170213
  140.90229167]
 ...
 [ 45.75404255  43.88108696  44.6        ...  50.48752525  50.93356902
   51.37961279]
 [ 51.82565657  52.27170034  52.71774411 ... 138.93276596 143.40630435
  121.17255319]
 [110.33723404 104.30673913  91.23234043 ... 127.56148936 110.74891304
  104.42425532]] [[ 91.72340426  83.6873913   73.0176087   71.40276596  63.32391304
   57.26369565  55.43        87.14765957  79.97543478  59.51565217
   55.73826087  53.37478261  43.31        38.89108696 127.61
   50.65195652  41.10086957  41.55085106  39.91413043  65.34065217
   56.22673913  58.12893617  49.58891304  28.56978261]
 [ 34.15531915  37.37695652  38.66130435  44.3506383   46.10195652
   40.68347826  49.78617021  46.27173913 137.922

[Parallel(n_jobs=64)]: Using backend MultiprocessingBackend with 64 concurrent workers.


i 0
mse 135.12967531246443
rmse 11.62452903615731
i 1
mse 94.74639518328404
rmse 9.733775998207687
i 2
mse 256.55833273117577
rmse 16.01743839479883
i 3
mse 210.9460946752438
rmse 14.523983430011334
i 4
mse 175.61921845356878
rmse 13.252140146163894
i 5
mse 245.43373226532935
rmse 15.666324784879489
i 6
mse 136.2596455834521
rmse 11.673030694016534
i 7
mse 275.69139140593643
rmse 16.603957100821972
i 8
mse 1115.3665295148505
rmse 33.39710360966727
i 9
mse 221.74255761328394
rmse 14.89102271884923
i 10
mse 288.7516898087501
rmse 16.992695189661646
i 11
mse 259.5932728614549
rmse 16.111898487188125
i 12
mse 368.6688812007798
rmse 19.200752099873064
i 13
mse 284.94592023652797
rmse 16.88034123578454
i 14
mse 924.3958570670491
rmse 30.40387898060129
i 15
mse 257.0533417741205
rmse 16.03288313978869
i 16
mse 319.4812107328747
rmse 17.87403733723511
i 17
mse 357.8345789982611
rmse 18.916516037533473
i 18
mse 185.48999949633495
rmse 13.619471336888775
i 19
mse 246.2401967019713
rmse 15.692042

[Parallel(n_jobs=64)]: Done   2 out of   4 | elapsed: 16.5min remaining: 16.5min
